# Preprocessing

- The LPIS field parcel data (geometries and class labels) are cleaned and preprocssed by a geopandas pipeline.
- The RGB-Sentinel-2 imagery is cut to small 128x128-pixel image chips. 
- The preprocessed LPIS data is clipped to matching chip geometries and saved in COCO annotation format. 

This repository only contains data for a small subset of the full Denmark region of interest for demonstration purposes. Imagery and geometry data for the full roi would take up several gigabytes. The full 2016 LPIS "marker" dataset set can be downloaded here:
https://kortdata.fvm.dk/download/Markblokke_Marker?page=MarkerHistoriske

![Subset roi](msc_codeshare/test_aoi_subset.png)

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from pathlib import Path

import utils
from utils.other import new_save, load_saved, load_or_new_save
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import rasterio
import shapely
import pprint

C:\Anaconda3\envs\geo_py37\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
fp_s2 = Path(r'data\RGB_small.tif')
fp_fields = Path(r'data\marker_small.shp')

### Vector preparation 

In [3]:
# Vector preparation

with rasterio.open(fp_s2) as src:
    meta = src.meta
    bounds = src.bounds

def prepare_vector(inpath, meta, bounds):
    df = (gpd.read_file(str(inpath), encoding='utf-8')  # contains danish special characters
             .rename(columns={'Afgroede': 'lc_name', 'AfgKode': 'lc_id', 'JOURNALNUM': 'journalnr'})
             .drop(['GB', 'GEOMETRISK', 'MARKNUMMER'], axis=1)
             .pipe(utils.geo.buffer_zero)
             .pipe(utils.geo.close_holes)
             .pipe(utils.geo.set_crs, 3044)
             .to_crs(meta['crs'])
             .pipe(utils.geo.clip, clip_poly=shapely.geometry.box(*bounds), explode_mp_=True)
             .pipe(utils.geo.reclassify_col, rcl_scheme=utils.geo.reclass_legend, 
                   col_classlabels='lc_name', col_classids='lc_id', drop_other_classes=True)
             .assign(geometry=lambda _df: _df.geometry.simplify(5, preserve_topology=True))
             .pipe(utils.geo.buffer_zero)
             .assign(area_sqm=lambda _df: _df.geometry.area)
             .pipe(utils.geo.reduce_precision, precision=4)
             .reset_index(drop=True)
             .assign(fid=lambda _df: range(0, len(_df.index)))
             .filter(['journalnr', 'lc_id', 'lc_name', 'rcl_lc_id', 'rcl_lc_name', 'area_sqm', 
                      'fid', 'geometry'])
          )
    return df


df = load_or_new_save(path=Path(r'output\preprocessed\prepared_vector.pkl'),
                      default_data=prepare_vector,
                      callable_args={'inpath': fp_fields, 'meta': meta, 'bounds': bounds})

print('df.info()', df.info())
display(df.head(3))

Loading from pickle file... prepared_vector.pkl
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 8 columns):
journalnr      113 non-null object
lc_id          113 non-null int64
lc_name        113 non-null object
rcl_lc_id      113 non-null int64
rcl_lc_name    113 non-null object
area_sqm       113 non-null float64
fid            113 non-null int32
geometry       113 non-null object
dtypes: float64(1), int32(1), int64(2), object(4)
memory usage: 6.7+ KB
df.info() None


,journalnr,lc_id,lc_name,rcl_lc_id,rcl_lc_name,area_sqm,fid,geometry
0,16-0036135,30,Ærter,5,other,18409.058038,0,"POLYGON ((513952.7684 6243596.0941, 514026.304..."
1,16-0044708,1,Vårbyg,1,springcereal,343798.036407,1,"POLYGON ((515394.05 6247182.35, 515511.8944 62..."
2,16-0053716,260,Græs med kløver/lucerne,4,grassland,202990.290354,2,"POLYGON ((512418.2498 6244766.2459, 512430.503..."


### Cut chip images & geometries

In [4]:
# Cut chip images & geomtries

chip_width, chip_height = 128, 128

chips_stats = load_or_new_save(path=Path(r'output\preprocessed\cut_chips.pkl'),
                               default_data=utils.img.cut_chips,
                               callable_args={'img_path': fp_s2,
                                              'df': df,
                                              'chip_width': chip_width,
                                              'chip_height': chip_height})

print('len chips_stats', len(chips_stats))

Loading from pickle file... cut_chips.pkl
len chips_stats 9


### Export geomtries as COCOjson

In [5]:
# Export geomtries as COCO annotation format

outpath_cocojson_train = Path(r'output\preprocessed\annotations\train2016.json')
outpath_cocojson_val = Path(r'output\preprocessed\annotations\val2016.json')

if outpath_cocojson_train.exists() and outpath_cocojson_val.exists():
    cocojson_train = load_saved(outpath_cocojson_train, file_format='json')
    cocojson_val = load_saved(outpath_cocojson_val, file_format='json')
else:
    train_split, val_split = utils.geo.train_test_split_coco(chips_stats)
    stats_train, stats_val = utils.geo.apply_split_coco(chips_stats, train_split, val_split)
    cocojson_train = utils.geo.format_cocojson(stats_train, chip_width, chip_height), 
    cocojson_val = utils.geo.format_cocojson(stats_val, chip_width, chip_height)
         
    new_save(outpath_cocojson_train, cocojson_train, file_format='json')
    new_save(outpath_cocojson_val, cocojson_val, file_format='json')

Loading from json file... train2016.json
Loading from json file... val2016.json


### Statistics

In [6]:
# Gather chip statistics.

def gather_statistics():
    return {
        'nr_chips': len(chips_stats.keys()),
        'nr_chips_train': len(train_split),
        'nr_chips_val': len(val_split),
        'nr_polys': sum([len(df['chip_df']) for df in chips_stats.values()]),
        'avg_polys_per_chip': sum([len(df['chip_df']) for df in chips_stats.values()]) / len(chips_stats.keys()),
        'nr_polys_train': sum([len(df['chip_df']) for df in [chips_stats[key] for key in train_split]]),
        'nr_polys_val': sum([len(df['chip_df']) for df in [chips_stats[key] for key in val_split]]),
        'train_rgb_mean': list(
            np.asarray([df['mean'] for df in [chips_stats[key] for key in train_split]]).mean(axis=0)),
        'train_rgb_stdn': list(np.asarray([df['std'] for df in [chips_stats[key] for key in val_split]]).mean(axis=0)),
        'fields per class_before_cut': df.groupby(['rcl_lc_name']).fid.aggregate(len).sort_values(ascending=False).to_dict(),
    }

statistics = load_or_new_save(path=Path(r'output\preprocessed\statistics.json'),
                              default_data=gather_statistics,
                              file_format='json')
pprint.pprint(statistics)

Loading from json file... statistics.json
{'avg_polys_per_chip': 19.0,
 'fields per class_before_cut': {'grassland': 40,
                                 'maize': 20,
                                 'other': 7,
                                 'springcereal': 32,
                                 'wintercereal': 14},
 'nr_chips': 9,
 'nr_chips_train': 7,
 'nr_chips_val': 2,
 'nr_polys': 171,
 'nr_polys_train': 123,
 'nr_polys_val': 48,
 'train_rgb_mean': [94.87407575334821, 106.43259974888393, 117.06084333147321],
 'train_rgb_stdn': [27.85198125213727, 15.87354983685378, 13.678598809683056]}


### Visualize preprocessed chip images and COCO annotations

In [7]:
in_json = outpath_cocojson_val
in_folder = r"output\preprocessed\image_chips\val2016"

In [12]:
geometries = utils.coco.coco_to_shapely(in_json, categories=[1])
geometries

Loading from json file... val2016.json


{'COCO_val2016_000000100005.jpg': <shapely.geometry.multipolygon.MultiPolygon at 0x18b5df606a0>,
 'COCO_val2016_000000100006.jpg': <shapely.geometry.multipolygon.MultiPolygon at 0x18b5df60390>}